In [3]:
class SCDEntry:
    def __init__(self):
        self.base_addr = 0   ###### 48-bit 这行稀疏存储的起始地址 
        self.ss_start  = 0   ###### 48-bit row_ptr[i] 
        self.ss_end    = 0   ###### 48-bit row_ptr[i+1]
        self.lil       = 0   ###### 48-bit 上一次的col_indices[j]
        self.stride    = 0   ###### 3-bit  多少stride表示一次跳跃多少字节
        self.valid     = 0   ###### 1-bit 是否检测到（标记该PC的间接加载模式）
        self.vector_size = 0  ###### 7-bit 打包的大小

    def set_vector_size(self, vector_size):
        self.vector_size = vector_size

    def set_base_addr(self, base_addr):
        self.base_addr = base_addr

    def set_ssb(self, ss_start, ss_end):
        self.ss_start = ss_start
        self.ss_end = ss_end

    def set_lil(self, lil):
        self.lil = lil

    def set_stride(self, stride):
        self.stride = stride

    def set_valid(self, valid):
        self.valid = valid

    def __repr__(self):
        return (f"base_addr={self.base_addr},"
                f"ss_start={self.ss_start},"
                f"ss_end={self.ss_end},"
                f"lil={self.lil},"
                f"stride={self.stride},"
                f"valid ={self.valid}")


class SCD:
    def __init__(self, max_entries=2):
        self.entryA = SCDEntry()
        self.entryB = SCDEntry()

    def init_entries(self, stride, vector_size):
        self.entryA.set_valid(0)
        self.entryB.set_valid(0)
        self.entryA.set_stride(stride)
        self.entryB.set_stride(stride)
        self.entryA.set_vector_size(vector_size)
        self.entryB.set_vector_size(vector_size)

    def update_boundary(self, entryid, ss_start, ss_end):
        if entryid == 0:
            self.entryA.set_ssb(ss_start, ss_end)
        elif entryid == 1:
            self.entryB.set_ssb(ss_start, ss_end)
        else:
            print("error")

    def get_relative_addr(self, entryid, ):
        if entryid == 0: #要a的相对地址
            relative_addr = self.entryA.base_addr + self.entryB.lil + self.entryA.stride
        elif entryid == 1: #要b的相对地址
            relative_addr = self.entryB.base_addr + self.entryA.lil + self.entryB.stride
        else:
            print("error")

        return relative_addr
    
    def generate_vector_access(self, entryid, ptr):        
        if entryid == 0:
            entry = self.entryA
        elif entryid == 1:
            entry = self.entryB
        else:
            print("error")
            return []

        # Check if the new ptr is within the previously prefetched range
        if ptr >= entry.lil:
            # New ptr is outside of previously prefetched range, generate new vector access
            vector_access = []
            for i in range(entry.vector_size):
                addr = ptr + i
                if addr >= entry.ss_start and addr < entry.ss_end:
                    vector_access.append(addr)
                else:
                    vector_access.append(-1)  # Fill with -1 if out of boundary
            
            # Update lil to the end of the current prefetch
            entry.set_lil(ptr + entry.vector_size)
            print(f"vector{entryid}:{vector_access}")
            return vector_access
        else:
            # The new ptr is within the previously prefetched range
            print("has send on last vector prefetch")
            return []
        

    def __repr__(self):
        entries_repr = "\n".join([str(self.entryA), str(self.entryB)])
        return f"Sparse Chain Detector Entries :\n{entries_repr}"


if __name__ == "__main__":
    sd = SCD()

    sd.init_entries(stride=1, vector_size=4)
    
    test_cases = [
        # entryid, ss_start, ss_end, ptr
        (1,  0,  6,  0),
        (1,  0,  6,  1),
        (1,  0,  6,  2),
        (1,  0,  6,  3),
        (1,  0,  6,  4),
        (1,  0,  6,  5),
        (1,  0,  6,  6),
        (1,  7,  9,  7),
        (1,  7,  9,  8),
        (1,  7,  9,  9),
        (1, 10, 18, 10),
        (1, 10, 18, 11),
        (1, 10, 18, 12),
    ]

    print("Initial State:")
    print(sd)

    ###### 对每个测试用例进行检测
    for entryid, ss_start, ss_end, ptr in test_cases:
        sd.update_boundary(entryid, ss_start, ss_end)
        sd.generate_vector_access(entryid, ptr)

    ###### 打印最终的SCD的状态
    print("Final State:")
    print(sd)


Initial State:
Sparse Chain Detector Entries :
base_addr=0,ss_start=0,ss_end=0,lil=0,stride=1,valid =0
base_addr=0,ss_start=0,ss_end=0,lil=0,stride=1,valid =0
vector1:[0, 1, 2, 3]
has send on last vector prefetch
has send on last vector prefetch
has send on last vector prefetch
vector1:[4, 5, -1, -1]
has send on last vector prefetch
has send on last vector prefetch
has send on last vector prefetch
vector1:[8, -1, -1, -1]
has send on last vector prefetch
has send on last vector prefetch
has send on last vector prefetch
vector1:[12, 13, 14, 15]
Final State:
Sparse Chain Detector Entries :
base_addr=0,ss_start=0,ss_end=0,lil=0,stride=1,valid =0
base_addr=0,ss_start=10,ss_end=18,lil=16,stride=1,valid =0
